# **Mini Project 1**

0. Requirements:
   
   If you do not have the following packages installed, run the command below to install them.

In [9]:
# !pip install pandas
# !pip install numpy
# !pip install scikit-learn
# !pip install matplotlib
# !pip install seaborn
# !pip install nltk
# !pip install codecarbon
# !pip install shap

1. Data Preparation:
   
    Goal: Load and inspect the IMDb dataset containing movie reviews labeled with positive and negative sentiments.(https://ai.stanford.edu/%7Eamaas/data/sentiment/)
    
    Task: Read the dataset, store the reviews and their associated sentiments, and explore the dataset to understand its structure.

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import shap
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords # Importe la liste des "stop words" (mots vides) de la bibliothèque NLTK (Natural Language Toolkit)
from nltk.stem import PorterStemmer # Importe la classe PorterStemmer de NLTK. Le stemming est un processus qui consiste à réduire les mots à leur racine (stem)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from codecarbon import EmissionsTracker


NameError: name 'nltk' is not defined

In [22]:
# TASK 1: Data Preparation 
# Load the dataset from the local aclImdb folder

import pandas as pd
import os

def load_movie_reviews(data_folder):
    """
    Charge les critiques de films à partir d'une structure de dossiers
    (pos/ et neg/) et les renvoie sous forme de DataFrame Pandas.

    Args:
        data_folder: Le chemin vers le dossier principal contenant les
                     sous-dossiers 'pos' et 'neg'.

    Returns:
        Un DataFrame Pandas avec deux colonnes : 'review' (texte de la critique)
        et 'sentiment' ('pos' ou 'neg').
        Retourne None si une erreur se produit.
    """
    reviews = []
    sentiments = []

    for sentiment in ['pos', 'neg']:
        folder_path = os.path.join(data_folder, sentiment)  # Chemin complet vers pos/ ou neg/

        if not os.path.isdir(folder_path):
            print(f"Erreur : Le dossier '{folder_path}' n'existe pas.")
            return None

        for filename in os.listdir(folder_path):
            if filename.endswith(".txt"):  # Important: traiter seulement les fichiers .txt
                file_path = os.path.join(folder_path, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:  # 'utf-8' pour gérer les accents
                        review_text = f.read()
                        reviews.append(review_text)
                        sentiments.append(sentiment)
                except FileNotFoundError:
                    print(f"Erreur: Fichier '{file_path}' introuvable (improbable).")
                    return None  # Tu peux choisir de continuer ou d'arrêter ici
                except Exception as e:
                    print(f"Erreur lors de la lecture de '{file_path}': {e}")
                    return None

    # Crée le DataFrame Pandas
    df = pd.DataFrame({'review': reviews, 'sentiment': sentiments})
    return df


# === Utilisation de la fonction ===

# ADAPTE CE CHEMIN !!!
# 1. Charge les données pour l'entraînement 
data_directory = "database_less/train"  #  Si le dossier 'aclImdb' est dans le même répertoire que ton notebook.
movie_reviews_train_df = load_movie_reviews(data_directory)
# 2. Charge les données pour le test
data_directory = "database_less/test"
movie_reviews_test_df= load_movie_reviews(data_directory)

# 3. Vérifie que le chargement a réussi et affiche des informations
if movie_reviews_test_df is not None:
    print(movie_reviews_test_df.head())  # Premières lignes
    print("-" * 20)  # Séparateur
    print(movie_reviews_test_df.info())  # Infos générales
    print("-" * 20)
    print(movie_reviews_test_df['sentiment'].value_counts())  # Compte les sentiments
    print("-" * 20)
    print("\nExemple de critique :")  #Afficher une critique complète
    print(movie_reviews_test_df['review'][0]) #La première
    print("Sentiment associé:", movie_reviews_test_df['sentiment'][0])

                                              review sentiment
0  This was absolutely one of the best movies I'v...       pos
1  As others that have commented around the web.....       pos
2  The plot of this movie is set against the most...       pos
3  The cinematography is the film's shining featu...       pos
4  (This has been edited for space)<br /><br />Ch...       pos
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6250 entries, 0 to 6249
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     6250 non-null   object
 1   sentiment  6250 non-null   object
dtypes: object(2)
memory usage: 97.8+ KB
None
--------------------
sentiment
pos    3125
neg    3125
Name: count, dtype: int64
--------------------

Exemple de critique :
This was absolutely one of the best movies I've seen. <br /><br />Excellent performances from a marvelous A-List cast that will move you from smiles to laughter to tear

2. Text Preprocessing:
   
    Goal: Clean and preprocess the text data to remove noise and prepare it for analysis.
    
    Task: Remove unnecessary characters (e.g., HTML tags, punctuation), convert text to lowercase, and process words by removing stop words and stemming/lemmatizing them.

In [ ]:
# TASK 2: Text Preprocessing 
# Remove HTML tags

from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

def remove_html_bs(text):
    """Supprime les balises HTML (avec BeautifulSoup)."""
    try:
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text(separator=" ")
    except Exception as e:
        print(f"Erreur lors du nettoyage HTML : {e}")
        return ""

def remove_special_characters(text):
    """Supprime les caractères spéciaux et la ponctuation."""
    pattern = r"[^a-zA-ZÀ-ÖØ-öø-ÿ0-9\s]"
    cleaned_text = re.sub(pattern, " ", text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def convert_to_lowercase(text):
    """Convertit une chaîne de caractères en minuscules."""
    return text.lower()

def remove_stopwords(text):
    """Supprime les mots vides (stop words) en utilisant NLTK."""
    stop_words = set(stopwords.words('english'))  # Important: Utilise 'english'
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words) # Reconstruit la phrase

def apply_stemming(text):
    """Applique le stemming (PorterStemmer) de NLTK."""
    stemmer = PorterStemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

def apply_lemmatization(text):
    """Applique la lemmatisation avec WordNetLemmatizer de NLTK."""
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

def clean_reviews(df):
    """
    Nettoie un DataFrame de critiques (HTML, caractères spéciaux, minuscules, stop words, stemming/lemmatization).
    Modifie le DataFrame en place.
    """
    # Vérifie si le DataFrame est vide
    if df.empty:
        print("Erreur : Le DataFrame est vide. Impossible de le nettoyer.")
        return

    # Vérifie si la colonne 'review' existe
    if 'review' not in df.columns:
        print("Erreur : La colonne 'review' est absente du DataFrame.")
        return

    # Applique les fonctions de nettoyage, en séquence
    df['review'] = df['review'].apply(remove_html_bs)
    df['review'] = df['review'].apply(remove_special_characters)
    df['review'] = df['review'].apply(convert_to_lowercase)
    df['review'] = df['review'].apply(remove_stopwords)
    # df['review'] = df['review'].apply(apply_stemming)  # Optionnel : Stemming
    # df['review'] = df['review'].apply(apply_lemmatization) # Optionnel : Lemmatization

    # La fonction ne retourne rien, car elle modifie le DataFrame directement


# 2. Nettoyer les données (si le chargement a réussi)
if movie_reviews_test_df is not None:
    clean_reviews(movie_reviews_test_df)  # Nettoyage *en place*
    print(movie_reviews_test_df.head())
    print("\nExemple de review après le prétraitement complet :")
    print(movie_reviews_test_df['review'][5])

NameError: name 'WordNetLemmatizer' is not defined

3. Feature Extraction:

    Goal: Transform the cleaned text into numerical features for machine learning.
   
    Task: Use a vectorization technique such as TF-IDF to convert the text into a numerical matrix that captures the importance of each word in the dataset.

In [13]:
# TASK 3: Feature Extraction 


4. Model Training:

    Goal: Train a machine learning model to classify reviews based on their sentiment.
    
    Task: Split the dataset into training and testing sets, train a Logistic Regression model, and evaluate its performance on the test data.

In [14]:
# TASK 4: Model Training 

# TASK 8: Track emissions during model training


5. Model Evaluation:

    Goal: Assess the performance of your model using appropriate metrics.
    
    Task: Evaluate precision, recall, and F1-score of the Logistic Regression model. Use these metrics to identify the strengths and weaknesses of your system. Visualize the Confusion Matrix to better understand how well the model classifies positive and negative reviews. Additionally, test the model with a new review, preprocess it, make a prediction, and display the result. Example: test it with a new review such as:
    "The movie had great visuals, but the storyline was dull and predictable." The expected output might be: Negative Sentiment.

In [15]:
# TASK 5: Model Evaluation 

# Classification Report

# Confusion Matrix

# Plot the Confusion Matrix

# Test with a new review
review = "The movie had great visuals but the storyline was dull and predictable."


6. Hyperparameter Tuning:

    Goal: Optimize your Logistic Regression model by tuning its hyperparameters.
   
    Task: Use an optimization method to find the best parameters for your model and improve its accuracy.

In [16]:
# TASK 6: Hyperparameter Tuning 

# TASK 8: Track emissions during Hyperparameter Tuning


7. Learning Curve Analysis:

    Goal: Diagnose your model's performance by plotting learning curves.
   
    Task: Analyze training and validation performance as a function of the training set size to identify underfitting or overfitting issues.


In [17]:
# TASK 7: Learning Curve Analysis


9. Ethical Considerations and Explainability:

    Goal: Discuss the ethics in using and deploying your AI-based solution by investigating and implementing suitable explainability methods.
    
    Task: Understanding how a machine learning model makes predictions is crucial for ensuring transparency, fairness, and accountability in AI deployment. One of the widely used techniques for model explainability is SHAP (SHapley Additive exPlanations), which helps determine how much each feature (word) contributes to a prediction.
    In this task, you will use SHAP to analyze the impact of individual words on sentiment classification. This will allow you to visualize which words increase or decrease the probability of a positive or negative sentiment prediction. Additionally, discuss key aspects such as potential biases in the model, fairness in outcomes, and accountability in AI decision-making. You can find more information here: https://shap.readthedocs.io/en/latest/generated/shap.Explanation.html

In [18]:
# TASK 9: Ethical Considerations & Explainability

# Show SHAP summary plot with proper feature names


10. Deployment Considerations for Embedded Systems:

    Goal: Optimize and convert the trained logistic regression model for deployment on embedded systems like Arduino
    
    Task: To deploy the trained logistic regression model on a resource-constrained embedded system like an Arduino, we must optimize and convert the model into a format suitable for execution in an environment with limited memory and processing power. Since embedded systems do not support direct execution of machine learning models trained in Python, we extract the model’s learned parameters—namely, the weights and bias—after training. These parameters are then quantized to fixed-point integers to eliminate the need for floating-point calculations, which are inefficient on microcontrollers.
    Once quantization is applied, we generate a C++ .h header file containing the model’s coefficients and bias, formatted in a way that allows direct use within an Arduino sketch. The final model is optimized to perform inference using integer arithmetic, making it both lightweight and efficient for deployment on microcontrollers. You can find more information here: https://medium.com/@thommaskevin/tinyml-binomial-logistic-regression-0fdbf00e6765

In [19]:
# TASK 10: Deployment Considerations (Model Quantization & Export for Arduino)
# Extract weights and bias from the trained logistic regression model

# Apply quantization (convert to fixed-point representation)

# Generate C++ header file for Arduino

# Save the header file
